In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import logging
import os
from pprint import pformat

import numpy as np
import pandas as pd
from hydra import compose, initialize
import matplotlib.pyplot as plt
from omegaconf import OmegaConf

from physped.omegaconf_resolvers import register_new_resolvers
from physped.io.readers import trajectory_reader
from physped.io.writers import save_piecewise_potential
from physped.preprocessing.trajectories import preprocess_trajectories
from physped.core.functions_to_discretize_grid import learn_potential_from_trajectories
from physped.core.trajectory_simulator import simulate_trajectories
from physped.core.functions_to_select_grid_piece import evaluate_selection_point, evaluate_selection_range, get_index_of_the_enclosing_bin
from physped.visualization.plot_trajectories import plot_trajectories
from physped.visualization.plot_discrete_grid import plot_discrete_grid
from physped.visualization.plot_histograms import create_all_histograms, plot_multiple_histograms
from physped.visualization.plot_1d_gaussian_fits import learn_piece_of_potential_plot

In [ ]:
# Load configuration parameters
env_name = "single_paths"

with initialize(version_base=None, config_path="../conf", job_name="test_app"):
    config = compose(
        config_name="config",
        return_hydra_config=True,
        overrides=[
            f"params={env_name}", 
            "params.data_source=local",
            "params.grid.theta.min_multiple_pi=-0.75",
            "params.grid.theta.segments=4",
            # "params.grid.spatial_cell_size=0.5",
            ],
    )
    register_new_resolvers()

# set log level
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# set plot style
plt.style.use(Path.cwd().parent / "conf/science.mplstyle")

# change working directory
working_dir = config.hydra.run.dir
os.makedirs(working_dir, exist_ok=True)
os.chdir(working_dir)

In [ ]:
logging.info('CONFIGURATION FILE:\n' + pformat(OmegaConf.to_container(config), depth=1))

In [ ]:
logging.info('PARAMETERS:\n' + pformat(OmegaConf.to_container(config.params), depth=1))

In [ ]:
trajectories = trajectory_reader[env_name](config)
logging.info('\n' + pformat(trajectories.head()))

In [ ]:
preprocessed_trajectories = preprocess_trajectories(trajectories, config=config)

In [ ]:
plot_trajectories(preprocessed_trajectories, config, "recorded")

In [ ]:
piecewise_potential = learn_potential_from_trajectories(preprocessed_trajectories, config)

In [ ]:
config.params.input_ntrajs = len(preprocessed_trajectories.Pid.unique())
simulated_trajectories = simulate_trajectories(piecewise_potential, config)

In [ ]:
plot_trajectories(simulated_trajectories, config, "simulated")

In [ ]:
config = evaluate_selection_point(config)
config = evaluate_selection_range(config)

logging.info('\n' + pformat(OmegaConf.to_container(config.params.selection.range), depth=2))
# logging.info(pformat(dict(config.params.selection.range)))

In [ ]:
observables = ["xf", "yf", "uf", "vf"]
config.params.simulation.ntrajs = len(simulated_trajectories.Pid.unique())
histograms = create_all_histograms(preprocessed_trajectories, simulated_trajectories, config)
plot_multiple_histograms(observables, histograms, "PDF", config)

In [ ]:
plot_discrete_grid(config)

In [ ]:
learn_piece_of_potential_plot(config)   

In [ ]:
def calculate_potential(curvature, center, offset, value):
    return curvature * (value - center) ** 2 + offset


# Analytical parabolic potential
yrange = np.arange(-0.6, 0.6, 0.01)
px_to_mm = {"x": 3.9, "y": 4.1}
beta = 1.8
pot0 = 0.04
# A = 0.3
y_cent = 0.02
parabolic_potential = beta * (yrange - y_cent) ** 2 + pot0

# Get index for a point on the grid
# point = [0.45, -10, 0.6, 0, 3]
point = [0.4, -10, 0.6, 0, 3]
bin_index = []
for dim, value in zip(config.params.grid.bins, point):
    bin_index.append(get_index_of_the_enclosing_bin(value, config.params.grid.bins[dim]))
bin_index[3] = 0

cmap = ["C0", "C1", "C2", "C3"] * 100
fig, ax = plt.subplots(figsize=(3.54, 1.5))
lw = 2

ybins = config.params.grid.bins.y
dy = ybins[1] - ybins[0]
middle_bins = ybins + dy / 2
for y_index in range(len(ybins) - 1):
    bin_index[1] = y_index
    # xmu, xvar, ymu, yvar, umu, uvar, vmu, vvar = piecewise_potential.fit_params[*bin_index, :]
    # if np.sum(piecewise_potential.fit_params[*bin_index, :]) == 0:
    #     continue

    offset = piecewise_potential.position_based_offset[bin_index[0], y_index]
    X_dashed = np.linspace(ybins[y_index] - dy / 2, ybins[y_index + 1] + dy / 2, 100)
    Vy_dashed = calculate_potential(
        piecewise_potential.curvature_y[*bin_index], piecewise_potential.center_y[*bin_index], offset, X_dashed
    )
    color = cmap[y_index]

    Vy_mid = calculate_potential(
        piecewise_potential.curvature_y[*bin_index],
        piecewise_potential.center_y[*bin_index],
        offset,
        middle_bins[y_index],
    )
    ax.plot(middle_bins[y_index], Vy_mid, color="w", marker="|", ms=3, zorder=20)
    ax.plot(X_dashed, Vy_dashed, alpha=0.4, linestyle="dashed", color=color, lw=lw)

    X_solid = np.linspace(ybins[y_index], ybins[y_index + 1], 100)
    Vy_solid = calculate_potential(
        piecewise_potential.curvature_y[*bin_index], piecewise_potential.center_y[*bin_index], offset, X_solid
    )
    ax.plot(X_solid, Vy_solid, color=color, lw=lw)

ax.set_xlim(config.params.default_ylims)
ax.grid(False)
ax.set_xticks(ybins)

y_walls = config.params.trajectory_plot.ywalls
# Plot grid
ax.vlines(ybins, 0, 1, lw=0.4, color="k", linestyle="dashed", alpha=0.6)
ax.hlines(np.linspace(0, 1, 6), y_walls[0], y_walls[1], lw=0.4, color="k", linestyle="dashed", alpha=0.6)

# Plot walls
ax.vlines(y_walls, 0, 2, "k")
for ywall in y_walls:
    if ywall < 0:
        fillbetweenx = [10 * ywall, ywall]
    elif ywall > 0:
        fillbetweenx = [ywall, 10 * ywall]
    ax.fill_between(
        fillbetweenx,
        2,
        0,
        color="k",
        alpha=0.3,
        zorder=30,
        hatch="//",
    )

plt.ylim(0, 0.6)
plt.ylabel("$U(y\\,|\\,\\Phi) + O(\\Phi)$")
plt.ylabel("$U(y\\,|\\vec{x}_s, \\vec{u}_s) + O(\\vec{x}_x, \\vec{u}_s)$")
plt.xlabel("y [m]")
plt.plot(
    yrange,
    parabolic_potential,
    "k--",
    lw=1.5,
    zorder=-20,
    label="Analytic potential \n$V(y) = \\beta y^2$ (Eq.~(6))",
)
plt.plot(
    yrange,
    parabolic_potential,
    "k--",
    lw=1.5,
    zorder=20,
    alpha=0.3,
    # label="Analytic potential \n$V(y) = \\beta y^2$ (Eq.~(5))",
)
plt.legend(loc="upper center")
# plt.savefig("../figures/potential_convolution_narrow_corridor.pdf")

In [ ]:
trajectories = pd.read_csv('glow19_pnasnexus.csv')
trajectories['time'] = pd.to_datetime(trajectories['time'])
# trajectories = trajectories[['time', 'tracked_object', 'x', 'y']].copy()
trajectories.head(3)

In [ ]:
path = trajectories['tracked_object'].unique()[0]

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1)
for rotation_rad in [0,np.pi/2,np.pi,np.pi*3/2]:
    rotation_degree = int(360 * rotation_rad / np.pi)
    ax.plot(
        rotation_degree/90,
        1,
        marker=(3, 0, rotation_degree),
        markersize=22,
        color='b',
        zorder=10,
    )

In [ ]:
path_pid = trajectories['tracked_object'].unique()[80]
path = trajectories[trajectories['tracked_object'] == path_pid].copy()
color = 'b'
path.head(2)
path["r"] = np.hypot(path["vx_sav"], path["vy_sav"])
path["theta"] = np.arctan2(path["vy_sav"], path["vx_sav"])
rotation_rad = path["theta"].iloc[0]

fig, ax = plt.subplots(1, 1)
for rowid in np.arange(1, len(path),5):
    rotation_rad = path["theta"].iloc[rowid] + np.pi/4
    rotation_degree = int(360 * rotation_rad / np.pi)
    ax.plot(
        path["x_sav"].iloc[rowid],
        path["y_sav"].iloc[rowid],
        marker=(3, 0, rotation_degree),
        markersize=10,
        color=color,
        zorder=10,
    )
    
ax.plot(path["x_sav"], path["y_sav"], color=color, lw=0.9, alpha=0.8, zorder=10)
ax.plot(path["x_sav"].iloc[0], path["y_sav"].iloc[0], color=color, lw=0.9, alpha=0.8, zorder=10, marker = 'o')

In [ ]:
# Update parameters
# cfg.params.dt = 0.0333333333333
cfg.params.fps = 30
cfg.params.colnames.xf = 'x_sav'
cfg.params.colnames.yf = 'y_sav'
cfg.params.colnames.Pid = 'tracked_object'
cfg.params.colnames.time = 'time'

# Infer the edge of the measurement domain from the data
xmin = int(np.floor(trajectories.x.min())) - 1
xmax = int(np.ceil(trajectories.x.max())) + 1
ymin = int(np.floor(trajectories.y.min())) - 1
ymax = int(np.ceil(trajectories.y.max())) + 1
cfg.params.trajectory_plot.xlims = [xmin, xmax]
cfg.params.trajectory_plot.ylims = [ymin, ymax]

In [ ]:
# Preprocess trajectories
preprocessed_trajectories = preprocess_trajectories(trajectories, config=cfg)
preprocessed_trajectories.head(3)

In [ ]:
cfg['params']

In [ ]:
plot_trajectories(preprocessed_trajectories, cfg, 'recorded')

In [ ]:
piecewise_potential = learn_potential_from_trajectories(preprocessed_trajectories, cfg)
save_piecewise_potential(piecewise_potential, Path.cwd())

In [ ]:
# Simulate new trajectories using the learned potential
simulated_trajectories = simulate_trajectories(piecewise_potential, cfg)
simulated_trajectories.head(3)

In [ ]:
plot_trajectories(simulated_trajectories, cfg, 'simulated')  

In [ ]:

cfg.params.histogram_plot.xlims = [xmin, xmax]
cfg.params.histogram_plot.ylims = [ymin, ymax]
cfg.params.histogram_plot.ylims = [ymin, ymax]
# Create histograms
observables = ["xf", "yf", "uf", "vf"]
histograms = create_all_histograms(preprocessed_trajectories, simulated_trajectories, observables = observables)

# Plot Histograms
plot_multiple_histograms(observables, histograms, "PDF", cfg)